In [118]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd

#stock index
# country='Vietnam'
country='New Zealand'

stockindex_raw=pd.read_excel(r'./Data/StockIndex.xls')
stockindex=stockindex_raw[['date','NZX50']]
stockindex.columns=['date','stockindex']
stockindex['date']=pd.to_datetime(stockindex['date'])
try:   
    stockindex['stockindex']=pd.to_numeric(stockindex['stockindex'].str.replace(",",''))
except:
    pass
stockindex_raw.tail(2)

,date,VNINDEX,STI,NZX50,ASX200
779,2022-12-05,1093.67,3267.54,4645.46,7325.604386
780,2022-12-06,1048.69,3252.37,4625.82,7291.266058


In [119]:
#google traffic
gtraffic=pd.read_csv(r'./Data/changes-visitors-covid.csv')
c1=gtraffic['Entity'].str.contains(country)
gtraffic['date']=pd.to_datetime(gtraffic['Day'])
gtraffic=gtraffic[c1]
gtraffic=gtraffic[['Entity', 'date', 'transit_stations', 'workplaces','retail_and_recreation']]
gtraffic.tail(2)

,Entity,date,transit_stations,workplaces,retail_and_recreation
81446,New Zealand,2022-10-14,-24.286,-4.429,12.143
81447,New Zealand,2022-10-15,-24.571,-4.143,12.429


In [120]:
owid_o=pd.read_csv(r'./Data/owid-covid-data.csv')
c1=owid_o['location'].str.contains(country)
owid=owid_o[c1].reset_index(drop=True)[['location', 'date','new_cases','new_deaths','stringency_index']]
owid['date']=pd.to_datetime(owid['date'])
owid.tail(2)

,location,date,new_cases,new_deaths,stringency_index
1010,New Zealand,2022-12-04,34502.0,23.0,NaN
1011,New Zealand,2022-12-05,3.0,NaN,NaN


In [121]:
df=owid.merge(gtraffic,on='date',how='left').merge(stockindex,on='date',how='left')
df=df.sort_values(by='date').reset_index(drop=True)
df

,location,date,new_cases,new_deaths,stringency_index,Entity,transit_stations,workplaces,retail_and_recreation,stockindex
0,New Zealand,2020-02-28,1.0,NaN,19.44,New Zealand,9.714,15.143,2.000,4820.59
1,New Zealand,2020-02-29,0.0,NaN,19.44,New Zealand,10.000,15.286,1.857,NaN
2,New Zealand,2020-03-01,0.0,NaN,19.44,New Zealand,10.857,15.286,1.286,NaN
3,New Zealand,2020-03-02,0.0,NaN,19.44,New Zealand,10.571,15.571,0.857,4753.07
4,New Zealand,2020-03-03,0.0,NaN,19.44,New Zealand,10.286,15.857,0.429,4857.04
...,...,...,...,...,...,...,...,...,...,...
1007,New Zealand,2022-12-01,11.0,0.0,NaN,NaN,NaN,NaN,NaN,4637.13
1008,New Zealand,2022-12-02,9.0,0.0,NaN,NaN,NaN,NaN,NaN,4632.07
1009,New Zealand,2022-12-03,8.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1010,New Zealand,2022-12-04,34502.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
# df['stockindex_7mean'] = df['stockindex'].rolling(7, 1).mean()
df['new_cases_7mean'] = df['new_cases'].rolling(7, 7).mean()
df['new_deaths_7mean'] = df['new_deaths'].rolling(7, 7).mean()
df['transit_stations_7mean'] = df['transit_stations'].rolling(7, 7).mean()
df['workplaces_7mean'] = df['workplaces'].rolling(7, 7).mean()
df['retail_and_recreation_7mean'] = df['retail_and_recreation'].rolling(7, 7).mean()


df=df[['date','location','stockindex', 'stringency_index', 'new_cases_7mean', 'new_deaths_7mean','new_cases','new_deaths',
         'transit_stations_7mean','workplaces_7mean', 'retail_and_recreation_7mean','transit_stations','workplaces','retail_and_recreation',
       ]]
df.to_clipboard(index=False)